In [4]:
import pandas as pd
!pip install psycopg2
import psycopg2
# !pip install sqlalchemy
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
# !pip install seaborn
import seaborn as sns
# !pip install pathlib
import pathlib



[notice] A new release of pip is available: 22.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# !pip install SQLAlchemy


[notice] A new release of pip is available: 22.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import sqlalchemy
sqlalchemy.__version__  

'2.0.9'

In [9]:
cred = {
    'host': 'ep-red-bar-a1rghkov.ap-southeast-1.aws.neon.tech',
    'user': 'siswa_bfp',
    'pass': 'bfp_aksel_keren',
    'db': 'ecommerce_db',
    'port': 5432
}

uri = f"postgresql+psycopg2://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri)
conn.connect()


In [7]:
from sqlalchemy import inspect
inspector = inspect(conn)

for table_name in inspector.get_table_names():
    print("Table: %s" % table_name)
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])
    print('\n')

Table: customers
Column: cust_id
Column: name
Column: phone
Column: address
Column: city
Column: province


Table: carts
Column: cart_id
Column: cust_id
Column: trx_date


Table: cartitems
Column: cart_item_id
Column: cart_id
Column: product_id
Column: order_amt


Table: products
Column: product_id
Column: name
Column: price




In [12]:
df_customers = pd.read_sql_table(table_name = 'customers',con = conn.connect())
df_carts = pd.read_sql_table(table_name = 'carts',con = conn.connect())
df_cartitems = pd.read_sql_table(table_name = 'cartitems',con = conn.connect())
df_products = pd.read_sql_table(table_name = 'products',con = conn.connect())


In [14]:
# tutup koneksi ke database
conn.dispose()

In [26]:
xlsx_url = 'https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/wrangling-bfp-aksel/refs/heads/main/ecommerce_data/Campaigns.xlsx' 
json_url = 'https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/wrangling-bfp-aksel/refs/heads/main/ecommerce_data/Products.json' 
df_campaign = pd.read_excel(xlsx_url)
df_catalogue = pd.read_json(json_url)

In [197]:
df_catalogue_products = pd.merge(df_products,df_catalogue,on=['product_id','name','price'])
table_products = df_catalogue_products[['product_id', 'name', 'price', 'description', 'category', 'image_url']]
table_products['category'] = table_products['category'].map(lambda x:x['category_id'])
table_products.rename(columns={'category':'category_id'},inplace=True)
table_products

C:\Users\msing\AppData\Local\Temp\ipykernel_23080\3582221351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_products['category'] = table_products['category'].map(lambda x:x['category_id'])
C:\Users\msing\AppData\Local\Temp\ipykernel_23080\3582221351.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_products.rename(columns={'category':'category_id'},inplace=True)


,product_id,name,price,description,category_id,image_url
0,1,Japchae,72500,A Korean dish made with stir-fried sweet potat...,1,https://fake.pacmann.io/img/1.jpg
1,2,Frozen Tater Tots,35000,"Small, grated potatoes formed into rounds and ...",2,https://fake.pacmann.io/img/2.jpg
2,3,Frozen Hot Pockets,32000,Microwaveable pastries filled with a variety o...,2,https://fake.pacmann.io/img/3.jpg
3,4,Duck Foie Gras,112500,A luxurious French delicacy made from the live...,3,https://fake.pacmann.io/img/4.jpg
4,5,Mee Goreng,66000,A flavorful and spicy stir-fried noodle dish f...,1,https://fake.pacmann.io/img/5.jpg
...,...,...,...,...,...,...
147,148,Chicken Quesadilla,108000,A Mexican dish made by stuffing a tortilla wit...,3,https://fake.pacmann.io/img/148.jpg
148,149,Egg Noodles with Stir-Fry Vegetables,64500,A simple Chinese dish made with soft egg noodl...,1,https://fake.pacmann.io/img/149.jpg
149,150,Frozen Breakfast Sandwich,53000,Pre-made breakfast sandwiches filled with eggs...,2,https://fake.pacmann.io/img/150.jpg
150,151,Blueberry Smoothie,21500,"A blend of blueberries, yogurt, and a touch of...",5,https://fake.pacmann.io/img/151.jpg


In [143]:
table_categories = pd.DataFrame(df_catalogue_products['category'].to_list()).drop_duplicates().reset_index(drop=True)[['category_id','name']]
table_categories

,category_id,name
0,1,noodles
1,2,frozen
2,3,chicken_duck
3,4,coffee
4,5,beverages
5,6,rice


In [209]:
df_tags = df_catalogue_products[['product_id','tags']]

tag_index = {value:str(key+1) for key,value in enumerate(sorted(df_tags.explode('tags')['tags'].unique()))}
df_tags = df_tags.explode('tags').reset_index(drop=True)
# df_tags['tag_id'] = df_tags['tags'].map(tag_index)
df_tags['tag_id'] = df_tags['product_id'].astype('str')
df_tags['tag_id'] = df_tags['tag_id'] +"-"+ df_tags['tags'].map(tag_index)
df_tags = df_tags.rename(columns={'tags':'tag'})
table_tags = df_tags[['tag_id','tag','product_id']]
table_tags



,tag_id,tag,product_id
0,1-280,potato,1
1,1-328,seasonings,1
2,1-369,stir-fried,1
3,1-185,korean,1
4,1-24,beef,1
...,...,...,...
656,152-72,comfort,152
657,152-136,food,152
658,152-57,cheese,152
659,152-82,creamy,152


In [142]:
table_campaigns = df_campaign[['campaign_id','name','campaign_type','start_date']]
table_campaigns


,campaign_id,name,campaign_type,start_date
0,1,Festival Makanan Sehat,Category,2023-01-01
1,2,Bulan Cita Rasa Nusantara,Product,2023-02-01
2,3,Mei Festival Makanan Lokal,Category,2023-05-01
3,4,Festival Pedas Juli,Category,2023-07-01
4,5,Bulan Makanan Manis,Category,2023-09-01
5,6,November Nikmat,Product,2023-11-01
6,7,Festival Makanan Laut Februari,Product,2024-02-01
7,8,Maret Rasa Pedas,Product,2024-03-01
8,9,Juni Hidangan Segar,Category,2024-06-01
9,10,Agustus Kuliner Kreatif,Product,2024-08-01


In [141]:
table_campaign_items = df_campaign[['campaign_id','campaign_type','campaign_objects']].copy()
table_campaign_items['campaign_objects'] = table_campaign_items['campaign_objects'].map(lambda x:x.split(';'))
table_campaign_items = table_campaign_items.explode('campaign_objects').reset_index(drop=True)
cat_mapper = dict(zip(table_categories['category_id'],table_categories['name']))
prd_mapper = dict(zip(df_products['product_id'],df_products['name']))
table_campaign_items['campaign_item'] = [cat_mapper[int(i[2])] if i[1]=='Category' else prd_mapper[int(i[2])] for i in table_campaign_items[['campaign_type','campaign_objects']].itertuples()]
table_campaign_items = table_campaign_items.reset_index(names='campaign_item_id').drop(columns=['campaign_objects'])
table_campaign_items

,campaign_item_id,campaign_id,campaign_type,campaign_item
0,0,1,Category,frozen
1,1,1,Category,chicken_duck
2,2,1,Category,rice
3,3,2,Product,Chicken Shawarma
4,4,2,Product,Frozen Hash Browns
5,5,2,Product,Iced Latte
6,6,2,Product,Duck Breast with Cherry Sauce
7,7,2,Product,Macchiato
8,8,2,Product,Frozen Lasagna
9,9,3,Category,noodles


### SQLAlchemy classes part

In [146]:
import hashlib
import typing
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import (
    declarative_base,
    mapped_column,
    relationship,
    sessionmaker,
    Mapped,
)

In [210]:
from __future__ import annotations
import hashlib
import typing
import sqlalchemy as sa
from typing import List
from typing import Optional
from sqlalchemy import create_engine
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy.orm import (
    DeclarativeBase,
    mapped_column,
    relationship,
    sessionmaker,
    Mapped,
)

cred = {
    'host': 'ep-square-pond-a5h66kcl.us-east-2.aws.neon.tech',
    'user': 'neondb_owner',
    'pass': 'Kd2w6OvIiUtr',
    'db': 'neondb',
    'port': 5432
}



uri = f"postgresql+psycopg2://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri,echo=True)
# conn.connect()


class Base(DeclarativeBase):
    pass


class Campaigns(Base):
    __tablename__ = "campaigns"

    campaign_id : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    campaign_type: Mapped[str] = mapped_column(sa.String)
    start_date  : Mapped[str] = mapped_column(sa.String(30))
    
    campitems_camp: Mapped[List['CampaignItems']] = relationship(back_populates='camp_campitems')

    # def __repr__(self) -> str:
    #     return f"<User(username={self.auth.username}, email={self.auth.email})>"

class CampaignItems(Base):
    __tablename__ = "campaign_items"

    campaign_item_id: Mapped[int] = mapped_column(primary_key=True)
    campaign_id     : Mapped[int] = mapped_column(ForeignKey("campaigns.campaign_id"))
    campaign_type   : Mapped[str] = mapped_column(sa.String)
    campaign_item   : Mapped[str] = mapped_column(sa.String)

    camp_campitems: Mapped['Campaigns'] = relationship(back_populates='campitems_camp')

class Products(Base):
    __tablename__ = "products"

    product_id  : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    price       : Mapped[int] = mapped_column(sa.Float)
    description : Mapped[str] = mapped_column(sa.String)
    category_id : Mapped[int] = mapped_column(ForeignKey('categories.category_id'))
    image_url   : Mapped[str] = mapped_column(sa.String)

    categories_product  : Mapped['Categories'] = relationship(back_populates='product_category')
    tags_product        : Mapped[List['Tags']] = relationship(back_populates='product_tags')
    cartitems_products  : Mapped[List['CartItems']] = relationship(back_populates='products_cartitems')

    # def __repr__(self) -> str:
    #     return f"<User(username={self.auth.username}, email={self.auth.email})>"

class Categories(Base):
    __tablename__ = "categories"

    category_id : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    
    product_category: Mapped[List['Products']] = relationship(back_populates='categories_product')

class Tags(Base):
    __tablename__ = "tags"

    tag_id      : Mapped[str] = mapped_column(primary_key=True)
    tag         : Mapped[str] = mapped_column(sa.String)
    product_id  : Mapped[int] = mapped_column(ForeignKey('products.product_id'))
    
    
    product_tags: Mapped['Products'] = relationship(back_populates='tags_product')


class Customers(Base):
    __tablename__ = "customers"

    cust_id     : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    phone       : Mapped[str] = mapped_column(sa.String(30))
    address     : Mapped[str] = mapped_column(sa.String)
    city        : Mapped[str] = mapped_column(sa.String(30))
    province    : Mapped[str] = mapped_column(sa.String(30))
    
    cart_customer  : Mapped[List['Carts']] = relationship(back_populates='customer_cart')
    

class Carts(Base):
    __tablename__ = "carts"

    cart_id     : Mapped[int] = mapped_column(primary_key=True)
    cust_id     : Mapped[int] = mapped_column(ForeignKey('customers.cust_id'))
    trx_date    : Mapped[str] = mapped_column(sa.String)
    
    customer_cart  : Mapped['Customers'] = relationship(back_populates='cart_customer')
    cartitems_carts  : Mapped[List['CartItems']] = relationship(back_populates='carts_cartitems')


class CartItems(Base):
    __tablename__ = "cart_items"

    cart_item_id: Mapped[int] = mapped_column(primary_key=True)
    cart_id     : Mapped[int] = mapped_column(ForeignKey('carts.cart_id'))
    product_id  : Mapped[int] = mapped_column(ForeignKey('products.product_id'))
    order_amt   : Mapped[int] = mapped_column(sa.Integer)
    
    carts_cartitems  : Mapped['Carts'] = relationship(back_populates='cartitems_carts')
    products_cartitems: Mapped['Products'] = relationship(back_populates='cartitems_products')



Base.metadata.create_all(conn)

2024-12-11 14:49:50,389 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-11 14:49:50,390 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:49:51,092 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-11 14:49:51,094 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:49:51,894 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-11 14:49:51,897 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:49:52,922 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-11 14:49:52,929 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [ ]:
cred = {
    'host': 'ep-square-pond-a5h66kcl.us-east-2.aws.neon.tech',
    'user': 'neondb_owner',
    'pass': 'Kd2w6OvIiUtr',
    'db': 'neondb',
    'port': 5432
}



uri = f"postgresql+psycopg2://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri,echo=True)


table_campaigns.to_sql(name = 'campaigns',con = conn,if_exists = "append",index = False)
table_campaign_items.to_sql(name = 'campaign_items',con = conn,if_exists = "append",index = False)
table_categories.to_sql(name = 'categories',con = conn,if_exists = "append",index = False)
df_customers.to_sql(name = 'customers',con = conn,if_exists = "append",index = False)
df_carts.to_sql(name = 'carts',con = conn,if_exists = "append",index = False)
table_products.to_sql(name = 'products',con = conn,if_exists = "append",index = False)
table_tags.to_sql(name = 'tags',con = conn,if_exists = "append",index = False)
df_cartitems.to_sql(name = 'cart_items',con = conn,if_exists = "append",index = False)


2024-12-11 14:50:01,408 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-11 14:50:01,411 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:50:01,951 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-11 14:50:01,953 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:50:02,638 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-11 14:50:02,639 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:50:03,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-11 14:50:03,361 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

871

In [196]:

# df_customers
# df_carts
# df_cartitems
table_products
# table_categories
# table_tags
# table_campaigns
# table_campaign_items

,product_id,name,price,description,category,image_url
0,1,Japchae,72500,A Korean dish made with stir-fried sweet potat...,1,https://fake.pacmann.io/img/1.jpg
1,2,Frozen Tater Tots,35000,"Small, grated potatoes formed into rounds and ...",2,https://fake.pacmann.io/img/2.jpg
2,3,Frozen Hot Pockets,32000,Microwaveable pastries filled with a variety o...,2,https://fake.pacmann.io/img/3.jpg
3,4,Duck Foie Gras,112500,A luxurious French delicacy made from the live...,3,https://fake.pacmann.io/img/4.jpg
4,5,Mee Goreng,66000,A flavorful and spicy stir-fried noodle dish f...,1,https://fake.pacmann.io/img/5.jpg
...,...,...,...,...,...,...
147,148,Chicken Quesadilla,108000,A Mexican dish made by stuffing a tortilla wit...,3,https://fake.pacmann.io/img/148.jpg
148,149,Egg Noodles with Stir-Fry Vegetables,64500,A simple Chinese dish made with soft egg noodl...,1,https://fake.pacmann.io/img/149.jpg
149,150,Frozen Breakfast Sandwich,53000,Pre-made breakfast sandwiches filled with eggs...,2,https://fake.pacmann.io/img/150.jpg
150,151,Blueberry Smoothie,21500,"A blend of blueberries, yogurt, and a touch of...",5,https://fake.pacmann.io/img/151.jpg
